In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df = pd.read_csv('testing_hour_prices2.csv', sep=';',decimal = ',')


In [3]:
class EV:
    def __init__(self, name, charge_init):
        self.name = name
        self.charge_init = charge_init
        self.charging_speed = 10
        self.price_memory = [[] for i in range(96)]
        self.average_price_memory = []
        self.cheapest_timesteps = []
    
    def fill_memory(self, current_hour, electricity_price):
        '''
        Fills the memory of agents with the previous prices
        
        SHOULD BE DONE ON SUPERCLASS LEVEL TO SAVE DATA AND COMPUTATIONS
        
        '''
        
        self.price_memory[(current_hour %96) - 1].append(electricity_price)
        
    def average_memory(self):
        '''
        From self.price_memory creates avarage prices for a 24h cycle
        
        Could be expanded to a 4*24h cycle if wanted
        
        '''
        self.average_price_memory = [round(np.mean(self.price_memory[i]),2) for i in range(len(self.price_memory))]
    
    

        
    def choose_cheapest_timesteps(self,starting_time,ending_time,charge_needed):
        '''This function will tell you the most economic (cheap) way of getting to a full charge within the time window, if possible
           The start and end time are ticks of 1 hour atm
           Charge needed still abstract/dimensionless, the amount of energy the car needs e.g. full or like 75% idc
           
           
           Function use:
           input starting and ending time of charge 
           function outputs cheapest predicted hours (ticks count of hour)
           hours can be set to charging? = true using this
        '''
        if starting_time%96 < ending_time%96:
            total_time_window = self.average_price_memory[starting_time%96:ending_time%96] #e.g. charging from 1AM to 3PM is from 1:00 - 3:00
        else:
            total_time_window = self.average_price_memory[starting_time%96:] + self.average_price_memory[:ending_time%96]
        timesteps_needed = math.ceil(charge_needed/self.charging_speed)
        if timesteps_needed > (abs(ending_time-starting_time)):
            # charge all the available times
            self.cheapest_timesteps = [i for i in range(starting_time,ending_time)]
        else:
            # give all indexes + starting_time that are cheapest
            timewindow = np.array(total_time_window.copy())
            idx = np.argpartition(timewindow, timesteps_needed)
            cheapest_timesteps = idx[:timesteps_needed].tolist()
            self.cheapest_timesteps = [i + starting_time for i in cheapest_timesteps]
        

In [4]:
EV1 = EV('test',20)

In [5]:
for i in range(200):
    EV1.fill_memory(df['Hour'][i], round(df['electricity_price'][i],2))

In [6]:
EV1.average_memory()

In [7]:
EV1.choose_cheapest_timesteps(21,25,0)

In [9]:
EV1.choose_cheapest_timesteps(110,200,65)

In [10]:
EV1.cheapest_timesteps

[174, 164, 112, 172, 159, 154, 176]

In [ ]:
    def choose_cheapest_timesteps(self,starting_time,ending_time,charge_needed):
        '''This function will tell you the most economic (cheap) way of getting to a full charge within the time window, if possible
           The start and end time are ticks of 1 hour atm
           Charge needed still abstract/dimensionless, the amount of energy the car needs e.g. full or like 75% idc
           
           
           Function use:
           input starting and ending time of charge 
           function outputs cheapest predicted hours (ticks count of hour)
           hours can be set to charging? = true using this
        '''
        if starting_time%96 < ending_time%96:
            total_time_window = self.average_price_memory[starting_time%96:ending_time%96] #e.g. charging from 1AM to 3PM is from 1:00 - 3:00
        else:
            total_time_window = self.average_price_memory[starting_time%96:] + self.average_price_memory[:ending_time%96]
        timesteps_needed = math.ceil(charge_needed/self.charging_speed)
        if timesteps_needed > (abs(ending_time-starting_time)):
            print('total time is insufficient to charge to full. Charging commencing immediately')
            return [i for i in range(starting_time,ending_time)]

        timewindow_copy = total_time_window.copy()
        timewindow_copy.sort()
        cheapest_values = timewindow_copy[:timesteps_needed]
        for i in cheapest_values:
            new_entry = total_time_window.index(i) + starting_time
            cheapest_starting_timesteps.append(new_entry)
            total_time_window.pop

        print('the cheapest hour to start are hours {} with a total value of {}'.format(cheapest_starting_timesteps,cheapest_values))